In [1]:
import os
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from subprocess import check_output 
import sys
from datetime import datetime
%matplotlib inline
os.chdir('C:/Kaggle/Airbnb')

In [52]:
countries=pd.read_csv('countries.csv')
sessions=pd.read_csv('sessions.csv')
test_users=pd.read_csv('test_users.csv')
train_users_2=pd.read_csv('train_users_2.csv')

In [53]:
# Preprocessing of train_users_2 (unique rows: 213451, uniqu columns: id)
#train_users_2.date_account_created=pd.to_datetime(train_users_2.date_account_created,format='%Y-%m-%d')
#train_users_2.date_first_booking=pd.to_datetime(train_users_2.date_first_booking,format='%Y-%m-%d')
#train_users_2.timestamp_first_active=train_users_2.timestamp_first_active.astype(str)
#train_users_2.timestamp_first_active=pd.to_datetime((train_users_2.timestamp_first_active.str.slice(0,4,1)+ '-' + train_users_2.timestamp_first_active.str.slice(4,6,1) + '-' + train_users_2.timestamp_first_active.str.slice(6,8,1)),format='%Y-%m-%d')
X_train=train_users_2[train_users_2['date_account_created'].str.slice(0,4,1).astype(int)>=2014]

In [54]:
#X_train.head()
X_test=test_users
X_test['country_destination']='NDF'
X_train=X_train[X_train['id'].isin(sessions['user_id'].unique().tolist())]
X_test=X_test[X_test['id'].isin(sessions['user_id'].unique().tolist())]
sessions=sessions.rename(columns={'user_id':'id'})
sessions=sessions[sessions['id'].isin(X_train['id'].unique().tolist()+X_test['id'].unique().tolist())]


In [55]:
#sessions.head()
df_action = pd.crosstab(index=sessions['id'], columns=[sessions['action']]).fillna(0)
df_action.reset_index(inplace=True)
df_action.iloc[:,1:]=df_action.iloc[:,1:].clip(0,1)
df_action.columns = ['id']+['action_' + str(col) for col in df_action.columns[1:]]

df_action_type = pd.crosstab(index=sessions['id'], columns=[sessions['action_type']]).fillna(0)
df_action_type.reset_index(inplace=True)
df_action_type.iloc[:,1:]=df_action_type.iloc[:,1:].clip(0,1)
df_action_type.columns = ['id']+['action_type_' + str(col) for col in df_action_type.columns[1:]]

df_action_detail = pd.crosstab(index=sessions['id'], columns=[sessions['action_detail']]).fillna(0)
df_action_detail.reset_index(inplace=True)
df_action_detail.iloc[:,1:]=df_action_detail.iloc[:,1:].clip(0,1)
df_action_detail.columns = ['id']+['action_detail_' + str(col) for col in df_action_detail.columns[1:]]

In [56]:
X_train=pd.merge(X_train.loc[:,('id','country_destination')],df_action,on='id',how='left')
X_train=pd.merge(X_train,df_action_type,on='id',how='left')
X_train=pd.merge(X_train,df_action_detail,on='id',how='left')
X_train.fillna(0,inplace=True)

X_test=pd.merge(X_test.loc[:,('id','country_destination')],df_action,on='id',how='left')
X_test=pd.merge(X_test,df_action_type,on='id',how='left')
X_test=pd.merge(X_test,df_action_detail,on='id',how='left')
X_test.fillna(0,inplace=True)

In [57]:
#Saving ids
train_id=X_train['id']
test_id=X_test['id']

In [58]:
X_train['target']=np.where(X_train['country_destination']=='NDF',0,1)
X_train.drop(['id','country_destination'], axis=1, inplace=True)
X_test.drop(['id','country_destination'], axis=1, inplace=True)
X_train=X_train.loc[:,X_train.sum()>4]
X_test=X_test.loc[:,X_train.columns]
X_test.drop(['target'], axis=1, inplace=True)

In [225]:
#LR process
#1: LR with all the features
#2: LR with higher IV features
#3: LR with chi square independence test w/ Target
#4: Check for multi collinearity (VIF for continous vars and chisquare test of top fields from #2 & #3 with each other)
#5: LR Forward/backward selection methods (sklearn doesn't have this)- use RFE
#6: LR with l1 and l2 norm

#Check and compare results of all 4 outputs
#Start with 70-30 split for train and validation -> get the final model and then cross vaidate

In [29]:
#X_train.target.value_counts()/X_train.target.count()
#http://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
#Select the features and run:
#Run1:runlr(X_train.iloc[:,:405], X_train.loc[:,'target'])
#Run2:runlr(X_train.loc[:,sorted(IV,key=IV.get,reverse=True)[:100]], X_train.loc[:,'target'])
#Run3:runlr(SelectKBest(chi2, k=100).fit_transform(X_train.iloc[:,:405], X_train.loc[:,'target']), X_train.loc[:,'target'])

#Check overlap b/w VIF and chisq
selector = SelectKBest(chi2, k=100);selector.fit(X_train.iloc[:,:405], X_train.loc[:,'target']);
varlist=list(set().union(sorted(IV(),key=IV().get,reverse=False)[:100],X_train.iloc[:,:405][selector.get_support(indices=True)].columns))
len(varlist)
#Union: 186 fields
var=list(set(X_train.iloc[:,:405][selector.get_support(indices=True)].columns) & set(sorted(IV(),key=IV().get,reverse=False)[:100]))
len(var)
#only 14 fields overlap

#Run4 (Top IV and VIF):runlr(X_train.loc[:,var], X_train.loc[:,'target'])





14

In [30]:
#Checking chi sqaure independence test of 186 fields
from scipy.stats import chi2_contingency
len(varlist)
chi={}
a1=[]
for col in varlist:
    print(col)
    chi[col]=[]
    for v in np.arange(0,len(varlist),1):
        #print(v)
        if varlist.index(col)==v:
            chi[col].append(col)
        else:
            if chi2_contingency(np.array(pd.crosstab(X_train.loc[:,col],X_train.loc[:,varlist[v]])))[1]==0:
                chi[col].append(varlist[v])
                
feimp={}
for col in varlist:
    feimp[col]=chi2_contingency(np.array(pd.crosstab(X_train.loc[:,col],X_train.loc[:,"target"])))[0]
    
feimp1=pd.DataFrame([[key,value] for key,value in feimp.items()],columns=["var","value"])
chi1=[]
for i in varlist:
    chi1.append(pd.merge(pd.DataFrame(chi[i],columns=["var"]),feimp1,how='left',on='var').value.max())
len(chi1)
final_var=pd.merge(pd.DataFrame(list(set(chi1)),columns=["value"]),feimp1,how='left',on='value')
vars=list(final_var.loc[:,"var"])
vars

action_home_safety_terms
action_domains
action_office_location
action_detail_unavailable_dates
action_detail_manage_listing
action_detail_view_search_results
action_references
action_detail_change_contact_host_dates
action_campaigns
action_overview
action_apply_reservation
action_detail_your_trips
action_detail_translations
action_edit
action_detail_listing_descriptions
action_qt2
action_complete_status
action_how_it_works
action_edit_verification
action_available
action_phone_verification
action_type_partner_callback
action_pending
action_detail_lookup_message_thread
action_12
action_detail_your_listings
action_detail_place_worth
action_verify
action_detail_create_listing
action_position
action_detail_user_wishlists
action_detail_similar_listings
action_currencies
action_phone_number_widget
action_qt_with
action_ajax_payout_edit
action_referrer_status
action_type_message_post
action_detail_listing_reviews_page
action_detail_airbnb_picks_wishlists
action_agree_terms_check
action_person

['action_detail_translations',
 'action_home_safety_landing',
 'action_how_it_works',
 'action_phone_verification',
 'action_set_password',
 'action_country_options',
 'action_detail_airbnb_picks_wishlists',
 'action_airbnb_picks',
 'action_detail_place_worth',
 'action_place_worth',
 'action_rate',
 'action_guarantee',
 'action_detail_host_guarantee',
 'action_show_code',
 'action_detail_create_phone_numbers',
 'action_reputation',
 'action_popular',
 'action_detail_popular_wishlists',
 'action_change_availability',
 'action_position',
 'action_detail_listing_reviews_page',
 'action_this_hosting_reviews',
 'action_host_summary',
 'action_tos_confirm',
 'action_ajax_payout_edit',
 'action_detail_your_listings',
 'action_relationship',
 'action_type_booking_request',
 'action_detail_change_trip_characteristics',
 'action_ajax_refresh_subtotal',
 'action_detect_fb_session',
 'action_referrer_status',
 'action_detail_header_userpic',
 'action_header_userpic',
 'action_host_2013',
 'action

In [32]:
# Run RFE using independent features vars (53 features)
vars
rfe = RFE(estimator=linear_model.LogisticRegression(C=1e5), n_features_to_select=1, step=1)
rfe.fit(X_train.loc[:,vars], X_train.loc[:,"target"])
print("Features sorted by their rank:")
print(sorted(zip(map(lambda x: round(x, 4), rfe.ranking_), vars)))
rfeout=pd.DataFrame(sorted(zip(map(lambda x: round(x, 4), rfe.ranking_), vars)))
rfeoutvars=list(rfeout.iloc[:,1])
rfeoutvars
# Run8: runlr(X_train.loc[:,rfeoutvars[:20]], X_train.loc[:,'target'])

# L1 regression with 53 vars
runlr(X_train.loc[:,rfeoutvars[:20]],X_train.loc[:,'target'])

# L2 regression with same 53 vars
#runlr(X_train.loc[:,rfeoutvars[:20]],X_train.loc[:,'target'])
#Default value is L2- just change the value of C paramters- High C->low regularisation. After fixing the features-choose C based on cross validation
# Select top features after RFE and evaluate
# Run L1 and L2 on above features and evaluate


Features sorted by their rank:
[(1, 'action_phone_verification'), (2, 'action_show_code'), (3, 'action_type_booking_request'), (4, 'action_change_availability'), (5, 'action_invalid_action'), (6, 'action_reputation'), (7, 'action_how_it_works'), (8, 'action_detail_listing_reviews_page'), (9, 'action_host_2013'), (10, 'action_type_message_post'), (11, 'action_relationship'), (12, 'action_airbnb_picks'), (13, 'action_detail_create_listing'), (14, 'action_position'), (15, 'action_detail_create_phone_numbers'), (16, 'action_rate'), (17, 'action_signed_out_modal'), (18, 'action_ajax_refresh_subtotal'), (19, 'action_place_worth'), (20, 'action_references'), (21, 'action_detail_airbnb_picks_wishlists'), (22, 'action_this_hosting_reviews'), (23, 'action_host_summary'), (24, 'action_country_options'), (25, 'action_confirm_email'), (26, 'action_detail_translations'), (27, 'action_detect_fb_session'), (28, 'action_detail_message_post'), (29, 'action_tos_confirm'), (30, 'action_detail_host_guarant

In [31]:
len(vars)

53

In [24]:
#LR function
from sklearn.linear_model import LogisticRegression
from sklearn import linear_model,datasets,metrics
from sklearn.cross_validation import train_test_split,cross_val_score
from sklearn.feature_selection import chi2, RFE,SelectKBest

def runlr(X,y):
    model=linear_model.LogisticRegression(C=1e5)
    model.fit(X,y)
    accu = cross_val_score(LogisticRegression(C=1e5), X, y, scoring='accuracy', cv=10)
    auc = cross_val_score(LogisticRegression(C=1e5), X, y, scoring='roc_auc', cv=10)
    preci = cross_val_score(LogisticRegression(C=1e5), X, y, scoring='precision', cv=10)
    recal = cross_val_score(LogisticRegression(C=1e5), X, y, scoring='recall', cv=10)
    print(" accuracy: " + str(accu.mean()) +" AUC: " + str(auc.mean()) + " percision: " + str(preci.mean()) + " recall: " + str(recal.mean()))
    #X_test['pred']=model.predict_proba(X_test)
    #print parameters, coeff,waldchisq,std.error
    #pd.concat([pd.DataFrame(X_train.loc[:,sorted(IV,key=IV.get,reverse=True)[:100]].columns),pd.DataFrame(np.transpose(model.coef_))], axis = 1).to_csv('./model.csv')
    #selector = SelectKBest(chi2, k=100);selector.fit(X_train.iloc[:,:405], X_train.loc[:,'target']);
    #pd.concat([pd.DataFrame(X_train.iloc[:,:405][selector.get_support(indices=True)].columns),pd.DataFrame(np.transpose(model.coef_))], axis = 1).to_csv('./model.csv')
    #pd.concat([pd.DataFrame(X_train.loc[:,var]),pd.DataFrame(np.transpose(model.coef_))], axis = 1).to_csv('./model.csv')
    pd.concat([pd.DataFrame(X_train.loc[:,rfeoutvars[:20]].columns),pd.DataFrame(np.transpose(model.coef_))], axis = 1).to_csv('./model.csv')
    
    
# calculating IV for all factors
def IV():
    IV={}
    for col in np.setdiff1d(X_train.columns,["target"]):
        a=pd.crosstab(X_train[col],X_train['target']).apply(lambda r: r/r.sum(), axis=1).loc[0,1]
        IV[col]=((2*a)-1)*np.log(a/(1-a))
    return(IV)

In [65]:
model=linear_model.LogisticRegression(C=1e5)
model.fit(X_train.loc[:,rfeoutvars[:20]],X_train.loc[:,'target'])
train_p=model.predict_proba(X_train.loc[:,rfeoutvars[:20]])
test_p=model.predict_proba(X_test.loc[:,rfeoutvars[:20]])

In [66]:
train_p=pd.DataFrame(pd.concat([pd.DataFrame(train_id),pd.DataFrame(train_p).iloc[:,1]],axis=1))
test_p=pd.DataFrame(pd.concat([pd.DataFrame(test_id),pd.DataFrame(test_p).iloc[:,1]],axis=1))

In [69]:
test_p.to_csv('./test_p.csv')
train_p.to_csv('./train_p.csv')

In [ ]:
#Part-2 Predicting country of booking

In [70]:
import pycrfsuite

ImportError: No module named 'pycrfsuite'